In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (20, 13)
%matplotlib inline
%config InlineBackend.figure_format = "retina"

In [8]:
rootpath="data_final_project/KuaiRec/"

big_matrix = pd.read_csv(rootpath + "data/big_matrix.csv")
#item_categories = pd.read_csv(rootpath + "data/item_categories.csv")
#item_daily_features = pd.read_csv("data/item_daily_features.csv")
#kuairec_caption_category = pd.read_csv("data/kuairec_caption_category.csv")
small_matrix = pd.read_csv(rootpath + "data/small_matrix.csv")
#social_network = pd.read_csv(rootpath + "data/social_network.csv")
#user_features = pd.read_csv("data/user_features.csv")

big_matrix.head()
small_matrix.head()

,user_id,video_id,play_duration,video_duration,time,date,timestamp,watch_ratio
0,14,148,4381,6067,2020-07-05 05:27:48.378,20200705.0,1.593898e+09,0.722103
1,14,183,11635,6100,2020-07-05 05:28:00.057,20200705.0,1.593898e+09,1.907377
2,14,3649,22422,10867,2020-07-05 05:29:09.479,20200705.0,1.593898e+09,2.063311
3,14,5262,4479,7908,2020-07-05 05:30:43.285,20200705.0,1.593898e+09,0.566388
4,14,8234,4602,11000,2020-07-05 05:35:43.459,20200705.0,1.593899e+09,0.418364


In [9]:
big_matrix.head()

,user_id,video_id,play_duration,video_duration,time,date,timestamp,watch_ratio
0,0,3649,13838,10867,2020-07-05 00:08:23.438,20200705,1.593879e+09,1.273397
1,0,9598,13665,10984,2020-07-05 00:13:41.297,20200705,1.593879e+09,1.244082
2,0,5262,851,7908,2020-07-05 00:16:06.687,20200705,1.593879e+09,0.107613
3,0,1963,862,9590,2020-07-05 00:20:26.792,20200705,1.593880e+09,0.089885
4,0,8234,858,11000,2020-07-05 00:43:05.128,20200705,1.593881e+09,0.078000


In [ ]:
big_matrix = pd.DataFrame(big_matrix)
small_matrix = pd.DataFrame(small_matrix)

merged_matrix = pd.merge(small_matrix, big_matrix, on=['user_id', 'video_id'], how='left', indicator=True)

all_present = all(merged_matrix['_merge'] == 'both')

all_present

False

In [ ]:
missing_entries = merged_matrix[merged_matrix['_merge'] == 'left_only']

missing_entries

,user_id,video_id,play_duration_x,video_duration_x,time_x,date_x,timestamp_x,watch_ratio_x,play_duration_y,video_duration_y,time_y,date_y,timestamp_y,watch_ratio_y,_merge
0,14,148,4381,6067,2020-07-05 05:27:48.378,20200705.0,1.593898e+09,0.722103,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,14,183,11635,6100,2020-07-05 05:28:00.057,20200705.0,1.593898e+09,1.907377,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,14,3649,22422,10867,2020-07-05 05:29:09.479,20200705.0,1.593898e+09,2.063311,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,14,5262,4479,7908,2020-07-05 05:30:43.285,20200705.0,1.593898e+09,0.566388,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,14,8234,4602,11000,2020-07-05 05:35:43.459,20200705.0,1.593899e+09,0.418364,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4676565,7162,2267,11908,5467,NaN,NaN,NaN,2.178160,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4676566,7162,2065,11919,6067,NaN,NaN,NaN,1.964562,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4676567,7162,1296,16690,19870,NaN,NaN,NaN,0.839960,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4676568,7162,4822,11862,24400,NaN,NaN,NaN,0.486148,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [23]:
first_line_small = small_matrix.iloc[0]

# Use the correct column names from big_matrix
matching_lines = big_matrix[
                             (big_matrix['video_id'] == first_line_small['video_id'])]

matching_lines

,user_id,video_id,play_duration,video_duration,time,date,timestamp,watch_ratio
2524,1,148,12042,6067,2020-07-05 08:02:30.775,20200705,1.593907e+09,1.984836
5562,3,148,8172,6067,2020-08-01 14:13:43.624,20200801,1.596262e+09,1.346959
9281,5,148,6212,6067,2020-07-08 16:27:50.865,20200708,1.594197e+09,1.023900
13081,7,148,12494,6067,2020-07-05 06:31:26.451,20200705,1.593902e+09,2.059337
16232,8,148,10148,6067,2020-07-05 09:22:43.056,20200705,1.593912e+09,1.672655
...,...,...,...,...,...,...,...,...
12504713,7161,148,4742,6067,2020-07-06 18:11:55.344,20200706,1.594030e+09,0.781605
12504714,7161,148,4595,6067,2020-07-06 18:12:00.661,20200706,1.594030e+09,0.757376
12508030,7163,148,14155,6067,2020-07-05 17:48:50.934,20200705,1.593943e+09,2.333114
12517768,7169,148,13862,6067,2020-07-06 08:46:16.125,20200706,1.593996e+09,2.284820


In [5]:
# I do the most simple thing, I remove the rows with missing values
interactions = interactions.dropna()
# I also remove the duplicates
interactions = interactions.drop_duplicates()
# I also remove the rows with negative timestamps
interactions = interactions[interactions["timestamp"] >= 0]